In [1]:
import os
import pandas as pd
import numpy as np
from os import listdir
from os import path
import json
import matplotlib.pyplot as plt
import re
# % matplotlib inline

In [2]:
# i = 0
# for path in listdir('../data/tmp_JSON_labeled_commits/'):
#     print(i,path)
#     i += 1

#  file threshold

In [3]:
# parse json to files and actions

def parse_json(filepath_list):
    '''
    function used to parse json of each commit json file
    
    Args:
        filepath_list - list of filepaths
    
    Returns:
        files_json - list object contains parsed information
    
    '''
    number_of_files = []
    files_json = []
    
    # each commits
    for path in filepath_list:
        with open(path,'rb') as f:
            data = json.load(f)
            files_list = []
            # each file in commits
            for file in data['files']:
                # parse only cluster file
                for key in file.keys():
                    if re.match('^.*_cluster$',key):
                        actions_list = []
                        actions = file[key]['actions']
                        # each action in file
                        for action in actions:
                            actions_list.append(action['root'])
                        files_list.append(actions_list)
        
        files_json.append(files_list)        
    
    # return
    return files_json

# get file path list
filepath_list = ['../data/tmp_JSON_labeled_commits/' + path for path in listdir('../data/tmp_JSON_labeled_commits/')]
files_json = parse_json(filepath_list)

In [4]:
# getting file threshold
threshold = 0.95
number_of_files = [len(files) for files in files_json]

def get_file_threshold(number_of_files, threshold = 0.95):
    '''
    get padding threshold for files dimension
    
    Args:
        number_of_files - array of the number of files in each commits
        threshold - drop all commits with its the number of files beyond this threshold
    Returns:
        padding threshold - number
    '''
    
    total_files = len(number_of_files)
    number_of_files = np.array(number_of_files)
    bincount = np.bincount(number_of_files)

    sum_file = 0
    for index, item in enumerate(bincount):
        sum_file += item
        #print(index,item)
        #print(sum_file)
        if sum_file > threshold*total_files:
            padding_files_threshold = index
            break
            
    return padding_files_threshold

file_threshold = get_file_threshold(number_of_files, threshold)
print(file_threshold)

22


# root_threshold

In [5]:
threshold = 0.95
number_of_root = []
root_count = []
for files in files_json:
    number_of_root.extend([len(roots) for roots in files])
    root_count.append([len(roots) for roots in files])

In [6]:
# getting root threshold

def get_root_threshold(number_of_root, threshold = 0.95):
    '''
    get padding threshold for root dimension
    
    Args:
        number_of_root - array of the number of root in each commits
        threshold - drop all commits with its the number of roots beyond this threshold
    Returns:
        padding threshold - number
    '''
    
    total_root = len(number_of_root)
    number_of_root = np.array(number_of_root)
    bincount = np.bincount(number_of_root)

    sum_root = 0
    for index, item in enumerate(bincount):
        sum_root += item
        #print(index,item)
        #print(sum_file)
        if sum_root > threshold*total_root:
            padding_root_threshold = index
            break
            
    return padding_root_threshold

root_threshold = get_root_threshold(number_of_root, threshold)
print(root_threshold)

38


# select files which comply with the standard  
[roots number $\le$ 8, files number $\le$ 5]

In [7]:
file_threshold = 5
root_threshold = 8

In [8]:
cmt_used = []  # index of commits which comply with the standard.
i = 0
for counts in root_count:
    if (number_of_files[i] <= file_threshold) and (False not in [
            items <= root_threshold for items in counts
    ]):
        cmt_used.append(i)
    i += 1

In [9]:
# cmt_used

In [10]:
print("We will use {} commits while the total number of commits is {}".format(
    len(cmt_used), len(files_json)))

We will use 487 commits while the total number of commits is 918


#  Data preparation 

In [11]:
path = []
for index in cmt_used:
    path.append(filepath_list[index])
files_data = parse_json(path)

In [12]:
first_word = {}
counting = {}
counting_ins = {}
counting_update = {}
counting_mov = {}
counting_del = {}

for file_index, files in enumerate(files_data):
    #     print(">>>", file_index)
    for root_index, roots in enumerate(files):
        for action_index, actions in enumerate(roots):
            temp = actions.split(' at ')[0]
            first_word[temp.split()[0]] = first_word.get(temp.split()[0],
                                                         0) + 1
            tempq = []
            #             print(file_index,temp)
            if temp.startswith('INS'):
#                 if file_index == 226:
#                     print(temp)
                tempq.append('INS')
                words = [temp.split('INS ')[1].split(' to ')[0]
                         ] + [temp.split('INS ')[1].split(' to ')[-1]]
                for items in words:
                    #                     print(items)
#                     if file_index == 226:
#                         print(tempq)
                    items = items.split(': ')[0]
                    tempq.append(items)
                    counting_ins[items.strip()] = counting_ins.get(
                        items.strip(), 0) + 1
                    counting[items.strip()] = counting.get(items.strip(),
                                                           0) + 1
                temp = ' '.join(tempq)
                if file_index == 226:
                    print(tempq)
                    print(temp)

            if temp.startswith('UPDATE'):
                temp = 'UPDATE'
            if temp.startswith('MOVE'):
                temp2 = temp.split(' from ')[1]
                counting_mov[temp2.split(': ')[0]] = counting_mov.get(
                    temp2.split(': ')[0], 0) + 1
                counting[temp2.split(': ')[0]] = counting.get(
                    temp2.split(': ')[0], 0) + 1
                tempq.append('MOVE')
                tempq.append(temp2.split(': ')[0])
                temp = ' '.join(tempq)

            if temp.startswith('DEL'):
                #                 temp = temp
                counting_del[temp.split('DEL ')[1].split(': ')
                             [0]] = counting_del.get(
                                 temp.split('DEL ')[1].split(': ')[0], 0) + 1
                counting[temp.split('DEL ')[1].split(': ')[0]] = counting.get(
                    temp.split('DEL ')[1].split(': ')[0], 0) + 1
                tempq.append('DEL')
                tempq.append(temp.split('DEL ')[1].split(': ')[0])
                temp = ' '.join(tempq)
#                 print(temp)

            files_data[file_index][root_index][action_index] = temp
            #             print(root_index, action_index)

In [13]:
b = []
dic = {}
i = 0
for k,v in counting.items():
    i += 1
    b.append((v,k))
    dic[k] = i # encode strats with 1, because 0 means no word
b = sorted(b,reverse=True)
i = 0
for k,v in first_word.items():
    i += 1
    dic[k] = i

In [14]:
vec_ =np.zeros((len(files_data),root_threshold,3,file_threshold), dtype=int).tolist()
for file_index, files in enumerate(files_data):
    for root_index, roots in enumerate(files):
        for action_index, actions in enumerate(roots):
#             print(actions)
            temp =np.zeros(3, dtype=int).tolist()
            words = actions.split()
#             print(words)
            for index, word in enumerate(words):
#                 temp[index] = dic[words]
#                 print(file_index,root_index,index,action_index)
                vec_[file_index][action_index][index][root_index] = dic[word]

In [15]:
np.array(vec_).shape

(487, 8, 3, 5)

* 487 is the numbers of commits
* 8 is the numbers of roots in each file
* 3 is one of the action
* 5 is the numbers of files in each commit